# Problem 966 - Triangle Circle Intersection
Let $I(a, b, c)$ be the largest possible area of intersection between a triangle of side lengths $a, b, c$ and a circle which has the same area as the triangle.
For example $I(3, 4, 5) \approx 4.593049$ and $I(3, 4, 6) \approx 3.552564$.

Find the sum of $I(a, b, c)$ for integers $a, b, c$ such that $1 \le a \le b \le c \lt a + b$ and $a + b + c \le 200$.

Give your answer rounded to two digits after the decimal point.
## Solution.

In [76]:
from math import sqrt, pi, gcd
import math
import numpy as np
from scipy.optimize import minimize

In [77]:
def triangle_area(a, b, c):
    p  = (a+b+c)/2
    return sqrt(p*(p-a)*(p-b)*(p-c))

def radius(a, b, c):
    A = triangle_area(a, b, c)
    return sqrt(A/pi)

In [78]:
def d(X, Y):
    '''
    Distace between X and Y
    '''
    return sqrt((X[0]-Y[0])**2 + (X[1]-Y[1])**2)

def d_l(A, X, Y):
    '''
    Distance of point A from the infinite line passing through X and Y
    '''
    x0, y0 = A
    x1, y1 = X
    x2, y2 = Y

    dx = x2 - x1
    dy = y2 - y1

    num = abs(dy * x0 - dx * y0 + x2*y1 - y2*x1)
    den = sqrt(dx*dx + dy*dy)

    return num / den

In [79]:
def intersection_line_circle(A, B, O, R):
    x_a, y_a = A
    x_b, y_b = B
    x_o, y_o = O

    x_a -= x_o
    x_b -= x_o
    y_a -= y_o
    y_b -= y_o

    # solve |A + t(B-A)| = R
    dx = x_b - x_a
    dy = y_b - y_a
    a = dx**2 + dy**2
    b = 2*dx*x_a + 2*dy*y_a
    c = x_a**2 + y_a**2 - R**2

    delta = b**2 - 4*a*c

    if delta < 0:
        return []
    
    sqrt_delta = sqrt(delta)

    t1 = (-b - sqrt_delta) / (2*a)
    t2 = (-b + sqrt_delta) / (2*a)

    points = []
    for t in (t1, t2):
        if 0 <= t <= 1:     # intersection lies ON segment AB
            x = x_a + t*dx + x_o
            y = y_a + t*dy + y_o
            if (x,y) not in points:
                points.append((x, y))

    return sorted(points, key=lambda P: (P[0]-A[0])**2 + (P[1]-A[1])**2)

In [80]:
def arc_area(P, Q, O, R):
    x_p, y_p = P
    x_q, y_q = Q
    x_o, y_o = O
    
    px, py = x_p - x_o, y_p - y_o
    qx, qy = x_q - x_o, y_q - y_o

    cross = px*qy - py*qx
    dot   = px*qx + py*qy

    theta = math.atan2(cross, dot)

    return abs(0.5 * R**2 * theta)


def triangle_area_points(A, B, C):
    x1, y1 = A
    x2, y2 = B
    x3, y3 = C
    return 0.5 * abs(x1*(y2-y3) + x2*(y3-y1) + x3*(y1-y2))



In [81]:
def triangle_cap_circle_area(A, B, C, O, R):
    # Collect points along edges and circle intersections
    vertices = [A]

    pts = intersection_line_circle(A, B, O, R)
    vertices += pts
    vertices.append(B)

    pts = intersection_line_circle(B, C, O, R)
    vertices += pts
    vertices.append(C)

    pts = intersection_line_circle(C, A, O, R)
    vertices += pts
    vertices.append(A)

    # Sum the areas
    area = 0.0
    n = len(vertices)
    for i in range(n):
        P = vertices[i]
        Q = vertices[(i+1)%n]

        # midpoint test: inside circle?
        mx = 0.5*(P[0]+Q[0])
        my = 0.5*(P[1]+Q[1])
        inside = ((mx-O[0])**2 + (my-O[1])**2) <= R**2

        if inside:
            area += triangle_area_points(O, P, Q)
        else:
            area += arc_area(P, Q, O, R)

    return abs(area)


In [97]:
def intersection_area(a, b, c, x, y):
    d = gcd(a, b, c)
    if d != 1:
        return intersection_area(a//d, b//d, c//d, x, y) * d**2

    R = radius(a, b, c)
    O = (x, y)

    # Find triangle coordinates
    x_a = (a**2 - b**2 + c**2)/(2*a)
    y_a = sqrt(c**2 - x_a**2)

    A = (x_a, y_a)
    B = (0, 0)
    C = (a, 0)

    return triangle_cap_circle_area(A, B, C, O, R)    

In [111]:
import numpy as np
from scipy.optimize import minimize
from math import sqrt

def triangle_vertices(a, b, c):
    B = (0, 0)
    C = (a, 0)
    x_A = (b**2 + a**2 - c**2) / (2*a)
    y2 = b**2 - x_A**2
    y_A = sqrt(max(y2, 0))
    A = (x_A, y_A)
    return np.array(A), np.array(B), np.array(C)

def I(a, b, c, intersection_area_func):
    A, B, C = triangle_vertices(a, b, c)
    x0 = (A + B + C) / 3
    
    res = minimize(lambda P: -intersection_area_func(a, b, c, P[0], P[1]),
                   x0, method='Nelder-Mead',
                   options={'xatol':1e-9, 'fatol':1e-12, 'maxiter':1000})
    
    max_val = -res.fun
    return max_val, res.x


In [112]:
I(3,4,5, intersection_area)

(np.float64(4.593049357085353), array([1.94009381, 1.21139447]))

In [113]:
I(3,4,6, intersection_area)

(np.float64(3.9709166529262223), array([1.70061048, 2.75106849]))

In [101]:
ans = 0
for a in range(1, 200):            
    for b in range(a, 200 - a + 1):   
        c_min = b
        c_max = min(a + b - 1, 200 - a - b)

        if c_min > c_max:
            continue

        for c in range(c_min, c_max + 1):
            ans += I(a, b, c, intersection_area)[0]

ans

np.float64(29524145.20090142)

In [ ]:
29,524,145.20090142

In [ ]:
ans : 29,337,152.09